<a href="https://colab.research.google.com/github/ajinkyabhanudas/SIADS696/blob/dev/Dataset_setup_Ajinkya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook works on the first step of the train pipeline: data loading 
- The idea is to use the contents of this notebook as a makefile target so it can be the first step to run as we keep populating our datastore.








In [12]:
!git clone https://github.com/ajinkyabhanudas/SIADS696.git

Cloning into 'SIADS696'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 263 (delta 8), reused 2 (delta 2), pack-reused 221
Receiving objects: 100% (263/263), 13.92 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [13]:
import glob
import json
import os
import pandas as pd

In [14]:
try:
  os.chdir('SIADS696')
except:
  print("You're either already in the SIADS696 directory, or the path specified isn't accessible")

### We'll extract our dataset in two parts:
- first extract the overall stats data per channel and store it along with the channel name instead of the id for available channels. (`overall_stats_raw_df`)
- And second, extract the per-video data into another dataframe. (`raw_df`)

In [93]:
rootdir = 'data'
feature_list = ['publishedAt', 'title', 'channelId', 'description',
                'channelTitle', 'tags', 'categoryId', 'viewCount', 
                'likeCount', 'favoriteCount', 'commentCount', 'duration',
                'definition', 'contentRating', 'topicCategories']

ov_dict_list = []
ov_temp_dict = {}

dict_list = []
temp_dict = {}

for path in glob.glob(f'./{rootdir}/*/*'):
  try:
    with open(path, "r") as read_file:
      data = json.load(read_file)
    for channel_id in data.keys():
      ov_temp_dict = data[channel_id]["channel_statistics"]
      channel_name = list(data[channel_id]["video_data"].keys())[0]
      ov_temp_dict["channelName"] = data[channel_id]["video_data"][channel_name]["channelTitle"]
      ov_dict_list.append(ov_temp_dict)
      
      for video_info in data[channel_id]["video_data"].values():
        temp_dict = video_info
        dict_list.append(temp_dict)       
  except:
    pass

overall_stats_raw_df = pd.DataFrame(ov_dict_list)
overall_stats_raw_df.drop("hiddenSubscriberCount", axis=1, inplace=True)

raw_df = pd.DataFrame(dict_list)
raw_df = raw_df[feature_list]

### Next, let's do a quick check on the data and take a quick look at the data in both the dfs

In [97]:
raw_df.info()
raw_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28725 entries, 0 to 28724
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   publishedAt      28725 non-null  object
 1   title            28725 non-null  object
 2   channelId        28706 non-null  object
 3   description      28706 non-null  object
 4   channelTitle     28706 non-null  object
 5   tags             27887 non-null  object
 6   categoryId       28706 non-null  object
 7   viewCount        28677 non-null  object
 8   likeCount        28230 non-null  object
 9   favoriteCount    28705 non-null  object
 10  commentCount     28270 non-null  object
 11  duration         28703 non-null  object
 12  definition       28703 non-null  object
 13  contentRating    28703 non-null  object
 14  topicCategories  28142 non-null  object
dtypes: object(15)
memory usage: 3.3+ MB


,publishedAt,title,channelId,description,channelTitle,tags,categoryId,viewCount,likeCount,favoriteCount,commentCount,duration,definition,contentRating,topicCategories
20664,2016-10-02T10:59:20Z,5 FUN CANDY SCIENCE EXPERIMENTS,UCJcycnanWtyOGcz34jUlYZA,These 5 science candy experiments are fun and ...,IncredibleScience,"[CANDY EXPERIMENTS, COOL CANDY EXPERIMENTS, aw...",22,55673,557,0,61,PT12M21S,hd,{},"[https://en.wikipedia.org/wiki/Food, https://e..."
27293,2015-10-06T15:33:27Z,FIFA 16 'MESSI AND BALE' 50K PACK ON FUT DRAFT,UCdPambxHRj0kdFPNoJFM98A,FIFA 16 Ultimate Team Draft. How far do I get ...,George Benson Travel,"[hurderofbuffalo, hurderofbuffalo fifa 16, fif...",20,300368,5235,0,174,PT13M28S,hd,{},"[https://en.wikipedia.org/wiki/Sports_game, ht..."
18826,2020-02-16T22:00:03Z,What If Humans Tried Landing On The Sun,UC9uD-W5zQHQuAVT2GdcLCvg,"In 2018, NASA launched the Parker Solar Probe,...",Science Insider,"[Business Insider, Science Insider, outer spac...",28,223378,4835,0,844,PT5M54S,hd,{},[https://en.wikipedia.org/wiki/Knowledge]
19990,2011-11-27T18:00:00Z,The Hairy Ball Theorem,UCUHW94eEFW7hkUMVaZz4eDg,Ever tried to comb a hairy ball? Math says you...,minutephysics,"[hairy, ball, balls, comb, topology, million, ...",28,2465864,20145,0,2478,PT1M19S,hd,{},[https://en.wikipedia.org/wiki/Knowledge]
11832,2015-10-27T14:40:02Z,15 Minute Hip Opening Yoga Sequence,UCI9s9nFu2m3K2CvhO2QVfTg,In this quick but effective 15 minute hip open...,Candace Cabrera,"[yoga, yoga channel, yogabycandace, yoga class...",24,53849,657,0,51,PT18M54S,hd,{},[https://en.wikipedia.org/wiki/Lifestyle_(soci...


In [96]:
overall_stats_raw_df.info()
overall_stats_raw_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   viewCount        74 non-null     object
 1   subscriberCount  74 non-null     object
 2   videoCount       74 non-null     object
 3   channelName      74 non-null     object
dtypes: object(4)
memory usage: 2.4+ KB


,viewCount,subscriberCount,videoCount,channelName
1,24185621112,53500000,140,EminemMusic
51,1780857621,10300000,438,AsapSCIENCE
49,529711853,2330000,526,Science Insider
26,12405546281,14300000,156150,CNN
60,3344186489,12600000,1441,TKOR
